# Проверка работы Word2Vec

### Настраиваем корневую папку

In [9]:
import first_script

### Импорт нужных библиотек

In [10]:
from model import Word2Vec
from tokenizer import AutoTokenizer
import os

### Загружаем текст на котором будем обучать модель

In [11]:
# text = "Мама мыла раму, пока кошка ходила по улице, а папа ел хлеб. Дети гуляли по улице!"
text = ""
with open('../data/voina_i_mir.txt', 'r') as f:
    text = f.read()

### Загружаем предобученый токенизатор

In [12]:
tokenizer = AutoTokenizer("../tokenizer/tokenizer.bin")

### Превращаем текст в индексы

In [13]:
text_idxs = tokenizer.encode(text)

### Параметры модели

In [14]:
embed_dim = 256
window_size = 20
device = 'cuda'

### Параметры обучения

In [15]:
num_epochs = 10
batch_size = 2500
num_workers = os.cpu_count()

### Загружаем модель и обучаем

In [16]:
model = Word2Vec(embed_dim, window_size, len(tokenizer.words), device=device)
res = model.fit(text_idxs, batch_size, num_epochs, num_workers=num_workers)

Epoch [1/10, time: 0.665 minutes]:
                            Train Loss 14.534278, Train Perplexity 2051902.442
                            Test Loss 13.560227, Test Perplexity 774696.493
Epoch [2/10, time: 1.396 minutes]:
                            Train Loss 12.278866, Train Perplexity 215101.727
                            Test Loss 11.979067, Test Perplexity 159383.248
Epoch [3/10, time: 2.218 minutes]:
                            Train Loss 11.004128, Train Perplexity 60121.833
                            Test Loss 11.179769, Test Perplexity 71665.823
Epoch [4/10, time: 3.123 minutes]:
                            Train Loss 10.221460, Train Perplexity 27486.771
                            Test Loss 10.691587, Test Perplexity 43984.249
Epoch [5/10, time: 4.068 minutes]:
                            Train Loss 9.722807, Train Perplexity 16694.039
                            Test Loss 10.402971, Test Perplexity 32957.400
Epoch [6/10, time: 5.019 minutes]:
                          

### Тестируем метод K_Nearest: ищет k ближайших слов на основе эмбеддингов

Гиперпараметр use_cosine отвечает за метод поиска расстояния между эмбеддингами. В случае use_cosine=True используется косинусовое расстояние (нормировано от 0 до 1). Иначе - евклидово расстояние

Гиперпараметр k - кол-во ближайших слов, которые выводятся. Замечу, что первое слово - само слово, по которому искали ближайшие слова. Выводится для удобства.

In [17]:
word1 = 'Андрей'
word1_idx = tokenizer.encode(word1)
near1 = model.k_Nearest(word1_idx, k=10, use_cosine=True)
print(tokenizer.decode(near1))

['Андрей', 'чопорного', 'беспорядки', 'должная', 'складная', 'попало', 'совершенного', 'juger', 'всяком', 'далекий', 'плачется']


In [18]:
word1 = 'Андрей'
word1_idx = tokenizer.encode(word1)
near1 = model.k_Nearest(word1_idx, k=10, use_cosine=False)
print(tokenizer.decode(near1))

['Андрей', 'складная', 'долго', 'должная', 'с', 'колебался', 'помещики', 'плачется', 'фунта', 'заживала', '?"']


Гиперпараметр return_embed выводит эмбеддинги данных слов, в случае если return_embed=True, иначе не выводит

In [19]:
word1 = 'добрый'
word1_idx = tokenizer.encode(word1)
near1, embed = model.k_Nearest(word1_idx, k=10, return_embed=True)
print(tokenizer.decode(near1), embed.shape)

['добрый', 'имеющий', 'Ураааа', 'штука', 'паром', 'заснувших', 'Братцы', 'Михаил', 'когда', 'выделывающих', 'усилились'] torch.Size([11, 256])


### Одна из главных особенностей модели: удобные и крутые визуализации!

#### Визуализируем метрики обучения

In [20]:
Word2Vec.plot_metrics(*res)

#### Визуализируем слова на двумерной плоскости на основе их эмбеддингов

Возьмем слово добрый и его 10 ближайших слов

In [21]:
Word2Vec.plot_embeddings(tokenizer.decode(near1), embed)

Добавим слово которое не является ближайшим словом к данному. Посмотрим где оно будет отображено

In [22]:
import torch

In [23]:
fake_word = 'Андрей'
fake_word_idx = tokenizer.encode(fake_word)
fake_embed = model[fake_word_idx]

fake_embed = torch.cat((embed, fake_embed), dim=0)
fake_near = tokenizer.decode(near1) + [fake_word]
Word2Vec.plot_embeddings(fake_near, fake_embed)

Как видим, оно действительно дальше всех! Интересно, что все слова ниже слова добрый...